# Azure Chat Agents with Function Bridge

## Overview

This notebook demonstrates a powerful workflow pattern: **connecting agents through function executors**. This enables data enrichment and context injection between agent stages.

### Pipeline Structure:

```
Research Agent → Function Bridge → Final Editor Agent
                (Enrich with References)
```

### Workflow Flow:

1. **Research Agent** - Drafts an initial short answer
2. **Function Bridge** - Simulates external data fetch and injects additional context
3. **Final Editor Agent** - Incorporates the new information and produces polished output

### Key Concepts:

- **Function Executor Bridge**: Using `@executor` decorator for lightweight data transformation nodes
- **Context Injection**: Adding external knowledge between agent stages
- **Agent Chaining**: Multi-stage agent pipelines with intermediate processing
- **Conversation Continuity**: Preserving and extending conversation history
- **AgentExecutorRequest/Response**: Contract types for agent-to-agent communication

### Features Demonstrated:

✅ `@executor` decorator for function-style workflow nodes
✅ Consuming `AgentExecutorResponse` and forwarding `AgentExecutorRequest`
✅ Streaming `AgentRunUpdateEvent` events across multi-stage pipelines
✅ External knowledge integration into agent workflows

### Prerequisites:

- ✅ Azure OpenAI configured with required environment variables
- ✅ Azure CLI authentication (`az login` completed)

## Import Required Libraries

In [ ]:
import asyncio
from typing import Final

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentExecutorRequest,
    AgentExecutorResponse,
    AgentRunResponse,
    AgentRunUpdateEvent,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    executor,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
# Load environment variables from .env file
load_dotenv('../../.env')


## Define External Knowledge Base

### Simulated External References

In a real-world scenario, this could be:
- Database queries
- API calls to knowledge bases
- Document retrieval systems
- Web searches

For this demo, we use a simple dictionary to simulate external knowledge retrieval.

In [ ]:
# Simulated external content keyed by topic keywords
EXTERNAL_REFERENCES: Final[dict[str, str]] = {
    "workspace": (
        "From Workspace Weekly: Adjustable monitor arms and sit-stand desks can reduce "
        "neck strain by up to 30%. Consider adding a reminder to move every 45 minutes."
    ),
    "travel": (
        "Checklist excerpt: Always confirm baggage limits for budget airlines. "
        "Keep a photocopy of your passport stored separately from the original."
    ),
    "wellness": (
        "Recent survey: Employees who take two 5-minute breaks per hour report 18% higher focus "
        "scores. Encourage scheduling micro-breaks alongside hydration reminders."
    ),
}

print("✅ External knowledge base initialized")
print(f"   Available topics: {', '.join(EXTERNAL_REFERENCES.keys())}")

## Define Knowledge Lookup Function

This helper function searches for relevant external knowledge based on keywords in the user's prompt.

In [ ]:
def _lookup_external_note(prompt: str) -> str | None:
    """Return the first matching external note based on a keyword search."""
    lowered = prompt.lower()
    for keyword, note in EXTERNAL_REFERENCES.items():
        if keyword in lowered:
            return note
    return None

print("✅ Knowledge lookup function defined")

## Define Function Bridge Executor

### The Bridge Pattern

This function executor sits between two agents and:
1. Receives the draft response from the first agent
2. Looks up relevant external knowledge
3. Injects it as a follow-up user message
4. Forwards the enriched conversation to the next agent

### Function Signature:

- **Input**: `AgentExecutorResponse` - Complete response from previous agent
- **Context**: `WorkflowContext[AgentExecutorRequest]` - Sends request to next agent

### Key Operations:

1. Extract conversation history
2. Find original user prompt
3. Lookup external knowledge
4. Append as new user message
5. Forward enriched conversation

In [ ]:
@executor(id="enrich_with_references")
async def enrich_with_references(
    draft: AgentExecutorResponse,
    ctx: WorkflowContext[AgentExecutorRequest],
) -> None:
    """Inject a follow-up user instruction that adds an external note for the next agent."""
    # Extract conversation history
    conversation = list(draft.full_conversation or draft.agent_run_response.messages)
    
    # Find the original user prompt
    original_prompt = next(
        (message.text for message in conversation if message.role == Role.USER), 
        ""
    )
    
    # Lookup external knowledge based on the prompt
    external_note = _lookup_external_note(original_prompt) or (
        "No additional references were found. Please refine the previous assistant response for clarity."
    )

    # Create follow-up message with external context
    follow_up = (
        "External knowledge snippet:\n"
        f"{external_note}\n\n"
        "Please update the prior assistant answer so it weaves this note into the guidance."
    )
    
    # Append to conversation
    conversation.append(ChatMessage(role=Role.USER, text=follow_up))

    # Forward enriched conversation to next agent
    await ctx.send_message(AgentExecutorRequest(messages=conversation))

print("✅ Function bridge executor defined")

## Create Azure OpenAI Chat Client

In [ ]:
# Create the Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)

print("✅ Azure OpenAI Chat Client created")

## Create Research Agent

### Agent Role:

The Research Agent produces an initial draft with:
- Short, bullet-style format
- Two actionable ideas
- Labeled as "Initial Draft"

In [ ]:
research_agent = chat_client.create_agent(
    name="research_agent",
    instructions=(
        "Produce a short, bullet-style briefing with two actionable ideas. "
        "Label the section as 'Initial Draft'."
    ),
)

print("✅ Research Agent created")

## Create Final Editor Agent

### Agent Role:

The Final Editor Agent:
- Uses all conversation context (including external notes)
- Merges draft and external knowledge
- Produces concise recommendations under 150 words

In [ ]:
final_editor_agent = chat_client.create_agent(
    name="final_editor_agent",
    instructions=(
        "Use all conversation context (including external notes) to produce the final answer. "
        "Merge the draft and extra note into a concise recommendation under 150 words."
    ),
)

print("✅ Final Editor Agent created")

## Build the Workflow

### Three-Stage Pipeline:

```
research_agent → enrich_with_references → final_editor_agent
```

### Workflow Construction:

1. Add both agents to workflow
2. Add function bridge executor
3. Connect edges: research → bridge → editor
4. Set research agent as start point
5. Configure final editor to output complete response

In [ ]:
workflow = (
    WorkflowBuilder()
    .add_agent(research_agent, id="research_agent")
    .add_agent(final_editor_agent, id="final_editor_agent", output_response=True)
    .add_edge(research_agent, enrich_with_references)
    .add_edge(enrich_with_references, final_editor_agent)
    .set_start_executor(research_agent)
    .build()
)

print("✅ Workflow built successfully!")
print("\n   Pipeline: research_agent → enrich_with_references → final_editor_agent")

## Run the Workflow with Streaming

### Execution Flow:

1. User provides query about workspace wellness
2. **Research Agent** generates initial draft (streams tokens)
3. **Function Bridge** enriches with external knowledge (silent processing)
4. **Final Editor** incorporates everything and produces final output (streams tokens)

### Event Handling:

- `AgentRunUpdateEvent` - Real-time token streaming from agents
- `WorkflowOutputEvent` - Final complete response

In [ ]:
# User query
user_query = "Create quick workspace wellness tips for a remote analyst working across two monitors."

print(f"\n📝 User Query: {user_query}\n")
print("=" * 70)
print("\n🔄 Streaming Workflow Execution:\n")

# Stream the workflow execution
events = workflow.run_stream(user_query)

last_executor: str | None = None

async for event in events:
    if isinstance(event, AgentRunUpdateEvent):
        # Print agent name when switching executors
        if event.executor_id != last_executor:
            if last_executor is not None:
                print()  # New line after previous agent
            print(f"\n🤖 {event.executor_id}:", end=" ", flush=True)
            last_executor = event.executor_id
        
        # Stream token inline
        print(event.data, end="", flush=True)
    
    elif isinstance(event, WorkflowOutputEvent):
        print("\n\n" + "=" * 70)
        print("\n📤 Final Workflow Output:\n")
        response = event.data
        if isinstance(response, AgentRunResponse):
            print(response.text or "(empty response)")
        else:
            print(response if response is not None else "No response generated.")

print("\n" + "=" * 70)
print("\n✅ Workflow execution completed!")

## Expected Output

### Sample Execution Flow:

```
🤖 research_agent: 
**Initial Draft**

• Set up ergonomic dual monitor positioning with proper eye level alignment
• Take regular 5-minute breaks every hour to reduce eye strain

🤖 final_editor_agent:
**Workspace Wellness Tips for Remote Analysts**

For optimal health with a dual-monitor setup:

1. **Ergonomic Setup**: Use adjustable monitor arms and sit-stand desks to reduce 
   neck strain by up to 30%. Position monitors at proper eye level.

2. **Movement Breaks**: Take two 5-minute breaks per hour (research shows 18% higher 
   focus scores). Set reminders to move every 45 minutes.

3. **Hydration & Micro-breaks**: Schedule regular hydration reminders alongside 
   movement breaks for sustained productivity.

======================================================================
📤 Final Workflow Output:

[Complete AgentRunResponse with merged content]
```

### What Happened:

1. **Research Agent** created initial 2-point draft
2. **Function Bridge** detected "workspace" keyword and injected external knowledge about:
   - Adjustable monitor arms reducing neck strain
   - Movement reminders every 45 minutes
3. **Final Editor** merged both sources into comprehensive recommendations

## Key Takeaways

### Function Bridge Pattern

✅ **When to Use Function Bridges:**
- Data enrichment between agent stages
- External API calls or database lookups
- Format transformation
- Context injection
- Validation or filtering

✅ **Benefits:**
- Lightweight, focused processing
- Clear separation of concerns
- Reusable across workflows
- No agent overhead for simple transformations

### Agent Executor Contracts

**AgentExecutorResponse** (from agent):
```python
- full_conversation: list[ChatMessage]  # Complete conversation history
- agent_run_response: AgentRunResponse  # Agent's response details
```

**AgentExecutorRequest** (to agent):
```python
- messages: list[ChatMessage]  # Conversation to process
```

### Function Executor Pattern

**Basic Structure:**
```python
@executor(id="function_name")
async def function_name(
    input: InputType,
    ctx: WorkflowContext[OutputType]
) -> None:
    # Process input
    result = transform(input)
    # Send to next node
    await ctx.send_message(result)
```

**Advantages vs Custom Executor Class:**
- More concise for simple operations
- No class boilerplate
- Perfect for stateless transformations
- Easier to test

### Multi-Stage Agent Pipelines

**Pattern:**
```
Agent 1 → Function Bridge → Agent 2 → Function Bridge → Agent 3
```

**Use Cases:**
- Research → Fact-check → Editor
- Analyst → Data Enrichment → Summarizer
- Generator → Validator → Refiner

### Conversation Continuity

✅ **Preserving Context:**
```python
# Extract full conversation
conversation = list(draft.full_conversation or draft.agent_run_response.messages)

# Append new context
conversation.append(ChatMessage(role=Role.USER, text=new_context))

# Forward to next agent
await ctx.send_message(AgentExecutorRequest(messages=conversation))
```

**Benefits:**
- Maintains full conversation history
- Enables context-aware responses
- Supports iterative refinement

### External Knowledge Integration

**Real-World Implementations:**

| Source | Integration Method |
|--------|-------------------|
| **Databases** | SQL queries in bridge function |
| **APIs** | HTTP calls to external services |
| **Vector Stores** | Semantic search for relevant docs |
| **Web Search** | Real-time web searches |
| **Document Stores** | RAG (Retrieval Augmented Generation) |

### Comparison: Agent vs Function Executors

| Feature | Agent Executor | Function Executor |
|---------|---------------|------------------|
| **Purpose** | AI reasoning | Data transformation |
| **Complexity** | High (LLM-powered) | Low (code-based) |
| **Cost** | API calls required | Free (local processing) |
| **Speed** | Slower (LLM latency) | Fast (immediate) |
| **Use Case** | Creative tasks | Data manipulation |
| **Streaming** | Yes (token-by-token) | No (instant) |

### Design Patterns

**1. Enrichment Pattern** (this notebook):
```
Agent → [Enrich with Data] → Agent
```

**2. Validation Pattern**:
```
Agent → [Validate Output] → Agent (if needed)
```

**3. Format Transformation**:
```
Agent → [Convert Format] → Agent
```

**4. Multi-Source Aggregation**:
```
Agent → [Fetch Data A] → [Fetch Data B] → Agent
```

### Next Steps

Explore more advanced agent patterns:
- **Tool Calls with Feedback**: Interactive tool execution with user approval
- **Custom Agent Executors**: Building specialized agent behaviors
- **Human-in-the-Loop**: Workflows requiring user intervention
- **Reflection Pattern**: Self-critique and iterative improvement